## Table of contents:

1. [Importing libraries](#Imports)
2. [Data Cleaning](#Cleaning)
3. [Research](#Research)
4. [First EDA without Target Feature](#FirstEDA)
5. [Loading data with Target](#loading)

### Power Output
6. [EDA Power Output](#power)
7. [Feature Creation](#Engineering)
8. [Splitting Data as Time Series - Final results](#Final)
9. [Sensitivity Analysis](#sensitivity_analysis)
10. [Recursive Feature Elimination Cross Validation](#rfecv)
11. [Description of New Metrics](#desciptionofnewmetrics)
12. [Modelling](#Modelling)
13. [Models Tuning](#Tuning)
14. [Summary and Conclusions](#summary_power)

### Efficiency
15. [EDA for Efficiency](#eff)
16. [Log Transform for Solar Irradiance](#logt)
17. [Outlier detection and removal](#outlier)
18. [Modelling for Efficiency](#model_eff)
19. [Model Tuning for Efficiency](#model_tun)
20. [Summary and Conclusions for Efficiency](#summary_eff)

<a name="Imports"></a>
## 1. Imports

In [ ]:
# Ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# Data Manipulation
import pandas as pd
import numpy as np
from scipy import stats
import math

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.tree import plot_tree

import os
import time

# Feature Selection
from sklearn.feature_selection import RFECV

# Modelling
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredError

import keras_tuner as kt

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [ ]:
country_names = ['Antartica', 'Australia', 'Beijing', 'Berlin', 'Brasilia', 'Pretoria', 'Washington']
cols_irradiation = ['Clear sky GHI']
cols_meteorology = [' Date', 'Temperature', 'Wind speed']

# Dataset Preparation
Preparing two datasets into one for each country with relevant features

In [ ]:
# On investigating the file I noticed that the data has to be pre-processed as the explanatory rows will affect how pandas will
# read it
with open('meteorology/'+country_names[0]+'.csv', 'r') as f:
    print(f.read())

## Testing Idea

In [ ]:
# all '# text' rows except the last which contains the columns names should be removed 
open_file = open('irradiation/'+country_names[0]+'.csv', 'r')
read_file = open_file.read()

# The idea is that splitting by \n# will leave what I require as the last value
new_read = read_file.split('\n#')

In [ ]:
# keeping just the last value of that list
clean = new_read[-1]

# writing into another csv file
write_to_file = open('clean.csv', 'w')
write_to_file.write(clean)

write_to_file.close()


In [ ]:
test = pd.read_csv('clean.csv', sep = ';')

In [ ]:
# Creating a cleaner function

def cleaner(file_path, met_or_irr = 'met'):
    
    # opening and reading files
    open_file = open(file_path, 'r')
    read_file = open_file.read()

    # splitting by '\n#'
    new_read = read_file.split('\n#')
    
    # keeping just the last value of that list
    clean = new_read[-1]

    # writing into another csv file
    if met_or_irr == 'met':
        write_to_file = open('clean_met.csv', 'w')
        write_to_file.write(clean)
    else:
        write_to_file = open('clean_irr.csv', 'w')
        write_to_file.write(clean)
    write_to_file.close()


In [ ]:
# With the above test, it works fine. Looping all files and extracting required columns

for name in country_names:
    # first step is to clean both irradiation and meteorology files
    print('Cleaning datasets for {}======='.format(name))
    irrad = cleaner('irradiation/'+name+'.csv', met_or_irr= 'irr')
    meteor = cleaner('meteorology/'+name+'.csv',)
    
    # Next I read both saved files
    print('Reading Cleaned datasets and selecting all columns for {}======='.format(name))
    irrad_set = pd.read_csv('clean_irr.csv', sep = ';')
    meteor_set = pd.read_csv('clean_met.csv', sep = ';')
    
    print('no of rows in both sets is {} and {}'.format(len(irrad_set), len(meteor_set)))
    
    # dividing Irradiation columns by 8
    irrad_set = round(irrad_set.drop(' Observation period', axis= 1)/8, 4)
    
    # Concatenating both tables
    print('Joining tables to get extracted cols for {}======='.format(name))
    joined = pd.concat([meteor_set, irrad_set], axis = 1 )
    joined.drop('UT time', axis=1, inplace= True)
    #joined['Clear sky GHI'] = round(joined['Clear sky GHI']/12, 4)
    joined.rename({' Date':'Date'}, axis = 1, inplace = True)

    
    # Confirming data
    print('Confirming join for {}======='.format(name))
    display(joined)

    # downloading into a folder called 'extracted_files_raw'
    print('Saving tables to get extracted cols for {}======='.format(name))    
    joined.to_csv('recent_extracted_files_raw/'+name+'.csv', index= False)
    
    # Removing new created files so they can be created afresh for next country
    print('Removing both newly created files after {}======='.format(name))
    print('\n')
    os.remove('clean_irr.csv')
    os.remove('clean_met.csv')


In [ ]:
# Some other tweak

for name in country_names:
    # first step is to clean both irradiation and meteorology files
    print('Cleaning datasets for {}======='.format(name))
    irrad = cleaner('irradiation/'+name+'.csv', met_or_irr= 'irr')
    meteor = cleaner('meteorology/'+name+'.csv',)
    
    # Next I read both saved files
    print('Reading Cleaned datasets and selecting all columns for {}======='.format(name))
    irrad_set = pd.read_csv('clean_irr.csv', sep = ';')[cols_irradiation]
    meteor_set = pd.read_csv('clean_met.csv', sep = ';')[cols_meteorology]
    
    print('no of rows in both sets is {} and {}'.format(len(irrad_set), len(meteor_set)))
    
    # Concatenating both tables
    print('Joining tables to get extracted cols for {}======='.format(name))
    joined = pd.concat([meteor_set, irrad_set], axis = 1 )
    joined['Clear sky GHI'] = round(joined['Clear sky GHI']/8, 4)
    joined.rename({' Date':'Date'}, axis = 1, inplace = True)
    
    # Confirming data
    print('Confirming join for {}======='.format(name))
    display(joined)

    print('max_value for {} is {}'.format(name, max(joined['Clear sky GHI'])))
    print('\n')
    
    # downloading into a folder called 'needed'
    print('Saving tables to get extracted cols for {}======='.format(name))    
    joined.to_csv('recent_needed/'+name+'.csv', index= False)
    
    # Removing new created files so they can be created afresh for next country
    print('Removing both newly created files after {}======='.format(name))
    print('\n')
    os.remove('clean_irr.csv')
    os.remove('clean_met.csv')


# Combining 
Combining all countries into one dataset.

In [ ]:
# combining all countries with a column to indicate country into one data set
country_df = pd.read_csv('recent_needed/'+country_names[0]+'.csv')
country_df['Country'] = [country_names[0]]*len(country_df)

for n in range(1, (len(country_names))):
    
    df = pd.read_csv('recent_needed/'+country_names[n]+'.csv', parse_dates= ['Date'])
    df['Country'] = [country_names[n]]*len(df)
    
    # printing out all country names to confirm the code does the right thing
    print(n, country_names[n], df.shape)
    country_df = pd.concat([country_df, df], axis = 0)

<a name="Cleaning"></a>
## 2. Data Cleaning

In [ ]:
country_df.reset_index(drop=True, inplace= True)

#### Data Type check

In [ ]:
country_df.info()

In [ ]:
# Date should be in datetime format.
country_df['Date'] = pd.to_datetime(country_df['Date'])

In [ ]:
country_df.info()

#### Missing data check

In [ ]:
country_df.isna().sum().sum()

The dataset has no missing values

#### Duplicates Check

In [ ]:
dup = country_df.duplicated()

In [ ]:
country_df[dup]

There are no duplicate entries in the dataset

<a name="Research"></a>
## 3. RESEARCH

**What is Irradiance ?**

Solar irradiance is the power per unit area (surface power density) received from the Sun in the form of electromagnetic radiation in the wavelength range of the measuring instrument. Solar irradiance is measured in watts per square metre (W/m2) in SI units.

Solar irradiance is often integrated over a given time period in order to report the radiant energy emitted into the surrounding environment (joule per square metre, J/m2) during that time period. This integrated solar irradiance is called solar irradiation, solar exposure, solar insolation, or insolation.

Irradiance may be measured in space or at the Earth's surface after atmospheric absorption and scattering. Irradiance in space is a function of distance from the Sun, the solar cycle, and cross-cycle changes. **Irradiance on the Earth's surface additionally depends on the tilt of the measuring surface, the height of the Sun above the horizon, and atmospheric conditions.** 
The solar irradiance received by a particular location or body of water depends on **the elevation above sea level, the angle of the sun (due to latitude, season and time of day) and scattering elements such as clouds** 

The study and measurement of solar irradiance have several important applications, including the prediction of energy generation from solar power plants, the heating and cooling loads of buildings, climate modeling and weather forecasting, passive daytime radiative cooling applications, and space travel.

**What is a clear sky condition ?**

A clear sky condition is defined generally as the absence of visible clouds across the entire sky dome, and clear sky irradiance is the irradiance (e.g., GHI) occurring during these conditions. A more precise definition of clear sky irradiance relies to some extent on judgment.

Clear sky irradiance, in particular, the global horizontal irradiance (GHI), provides information about the maximum possible solar energy resource available at the location under consideration, which is crucial in estimating or forecasting the performance of solar energy technologies.

Direct irradiance is the part of the solar irradiance that directly reaches a surface; diffuse irradiance is the part that is scattered by the atmosphere; global irradiance is the sum of both diffuse and direct components reaching the same surface.


<a name="FirstEDA"></a>
## 4. First EDA without Target Feature

In [ ]:
## Let's see what range the data covers
print('Minimum and maximum date for each country in the dataset are') 
(country_df.groupby('Country')['Date'].agg([min, max]))

In [ ]:
country_df['Country'].value_counts()

The dataset covers exactly 3 years across each country.

#### Clear sky GHI

From research, the GHI values the global horizontal irradiance (GHI), provides information about the maximum possible solar energy resource available at the location. So, it's an important feature.

In [ ]:
# Summary Statistics of the Clear sky GHI values across countries
def twenty_fifth(data):
    return np.percentile(data, 25)
def seventy_fifth(data):
    return np.percentile(data, 75)

country_df.groupby('Country')['Clear sky GHI'].agg([min, max, twenty_fifth, seventy_fifth, np.mean, np.std]).sort_values('mean')

In [ ]:
# Bar plot of the Clear sky GHI which will show the average for the GHI
palette = {'Antartica':'b', 'Australia':'orange', 'Beijing':'r', 'Berlin':'dimgrey', 'Brasilia':'purple', 'Pretoria':'green',
           'Washington':'violet'}
palette1 = {'Antartica':'b', 'Australia':'b', 'Beijing':'b', 'Berlin':'b', 'Brasilia':'b', 'Pretoria':'b',
           'Washington':'b'}

order = country_df.groupby('Country', as_index= False)['Clear sky GHI'].mean().sort_values('Clear sky GHI', ascending= False)
plt.figure(figsize= (12,7))
ax = sns.barplot(x= 'Country', y='Clear sky GHI', data = country_df, palette= palette1, ci= 0, 
            order=order.Country)
ax.bar_label(ax.containers[0])
plt.ylabel('Clear sky GHI (w/m²)', fontsize = 12)
plt.xlabel('Country', fontsize = 12)
plt.title('Average Clear sky GHI across countries', fontsize = 16)
#plt.savefig('Average_Clear_sky_GHI.jpg', dpi= 300)

In [ ]:
# distribution of Clear sky GHI values.

plt.figure(figsize= (12,7))
sns.boxplot(x= 'Country', y='Clear sky GHI', data = country_df, palette= palette, order = order.Country)
plt.ylabel('Clear sky GHI (w/m²)', fontsize = 12)
plt.xlabel('Country', fontsize = 12)
plt.title('Distribution of Clear sky GHI across Countries', fontsize = 16)
plt.grid()
#plt.savefig('Boxplot_Clear_sky_GHI.jpg', dpi= 300)

In [ ]:
plt.figure(figsize= (12,7))
sns.swarmplot(x='Country', y='Clear sky GHI', data = country_df, palette= palette, size = 2, order = order.Country)
plt.ylabel('Clear sky GHI (w/m²)', fontsize = 12)
plt.xlabel('Country', fontsize = 12)
plt.title('Swarm Distribution of Clear sky GHI across Countries', fontsize = 16)
plt.show()
#plt.savefig('Swarmplot_Clear_sky_GHI.jpg', dpi= 300)

#### Temperature

In [ ]:
# Converting to degree celcius
country_df['Temperature'] = country_df['Temperature']-273

In [ ]:
# Below, we see the temperature seasonality across countries
plt.figure(figsize= (15,10))
sns.lineplot(x='Date', y='Temperature', data = country_df, hue= 'Country', palette= palette)

plt.ylabel('Temperature(°c)', fontsize = 12)
plt.xlabel('Date', fontsize = 12)
plt.title('Temperature Seasonality across Countries', fontsize = 16)
sns.despine()
plt.grid()
#plt.savefig('Temperature_seasonality.jpg', dpi= 300)


In [ ]:
# resampling into months to create a better plot

country_resamp = pd.read_csv('recent_needed/'+country_names[0]+'.csv')
country_resamp['Date'] = pd.to_datetime(country_resamp['Date'])
country_resamp.set_index('Date', inplace = True)
country_resamp = country_resamp.resample('MS')['Temperature', 'Clear sky GHI'].mean()
country_resamp = pd.DataFrame(country_resamp)
country_resamp['Country'] = [country_names[0]]*len(country_resamp)

for a in range(1, (len(country_names))):
    
    df = pd.read_csv('recent_needed/'+country_names[a]+'.csv', parse_dates= ['Date'])
    df.set_index('Date', inplace = True)
    df = df.resample('MS')['Temperature', 'Clear sky GHI'].mean()
    df = pd.DataFrame(df)
    df['Country'] = [country_names[a]]*len(df)
    
    # printing out all country names to confirm the code does the right thing
    print(a, country_names[a], df.shape)
    country_resamp = pd.concat([country_resamp, df], axis = 0)


In [ ]:
country_resamp = country_resamp.reset_index()
country_resamp['Temperature'] = country_resamp['Temperature']-273

In [ ]:
country_resamp

In [ ]:
plt.figure(figsize= (15,10))
sns.lineplot(x='Date', y='Temperature', data = country_resamp, hue= 'Country', palette= palette)
sns.scatterplot(x='Date', y='Temperature', data = country_resamp, hue= 'Country', palette= palette, legend= False)
sns.despine()
plt.ylabel('Temperature(°c)')
plt.xlabel('Date', fontsize = 12)
plt.title('Monthly Average Temperature Seasonality across Countries', fontsize = 16)
plt.grid()
#plt.savefig('Average_Temperature_seasonality.jpg', dpi= 300)

Observations from data
1. Antartica is a cold continent and as expected, we see a low temperature range. We notice a seasonality and see that we have the highest temperature towards the end of each year and beginning of the next from the graph. This is because Antarctica's summer is from October to February. 
2. Australia is the hottest continent and is usually hotter towards end of a year and beginning of next. Summer - the three hottest months December, January and February.
3. We notice a similar seasonality as in Australia and Antartica with Pretoria which is in Africa (The second hottest conitinent after Australia). The seasonality in Australia and Pretoria is the opposite of Washington, Berlin and Beijin as we notice Summer usually occurs from October to february in the former three and that period is winter for the latter three. 
4. Brasilia is the Second hottest country from the graph which is in South America the third Hottest continent in the world.
5. Washington, Berlin and Beijin have similar temperature seasonality. Which means Summer and other seasons occurs at around the same period. 

In [ ]:
# Summary statistics of the temperatures across the different countries
country_df.groupby('Country')['Temperature'].agg([min, max, np.mean]).sort_values('mean')

Rank from coldest to hottest from statistics and plots
1. Antartica
2. Beijing
3. Washington
4. Berlin
5. Pretoria
6. Brasilia
7. Australia

In [ ]:
# Temperature distribution in a box plot
plt.figure(figsize= (12,7))
sns.boxplot(x= 'Country', y='Temperature', data = country_df, palette= palette, order = order.Country)
sns.despine()
plt.ylabel('Temperature(°c)', fontsize = 12)
plt.xlabel('Country', fontsize = 12)
plt.title('Temperature distribution across Countries', fontsize = 16)
plt.grid()
#plt.savefig('Box_plot_Temperature.jpg', dpi= 300)

In [ ]:
# Seeing the temperature distributions in a swarm plot
plt.figure(figsize= (12,7))
sns.swarmplot(x='Country', y='Temperature', data = country_df, palette= palette, size = 2, order= order.Country)
sns.despine()
plt.ylabel('Temperature(°c)', fontsize = 12)
plt.xlabel('Country', fontsize = 12)
plt.title('Swarm Temperature distribution across Countries', fontsize = 16)
plt.grid()
#plt.savefig('Swarmplot_Temperature.jpg', dpi= 300)

We clearly see the differnce in temperature range across countries with this plot

#### Exploring relationship between Temperature and Clear Sky GHI

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Temperature', y='Clear sky GHI', data = country_df, hue= 'Country', palette = palette)
plt.xlabel('Temperature(°c)', fontsize = 12)
plt.ylabel('Clear sky GHI (w/m²)', fontsize = 12)
plt.title('Temperature(°c) vs Clear sky GHI (w/m²)', fontsize = 16)
plt.grid()
plt.savefig('Temperature_vs_ClearskyGHI.jpg', dpi= 300)

Observations
1. Generally, there's a postive relationship between temperature and Clear sky GHI.

In [ ]:
# Showing each country seperately

sns.set_style('whitegrid')
plt.figure(figsize= (8,5))
sns.relplot(x='Temperature', y='Clear sky GHI', data = country_df, kind= 'scatter', col= 'Country', col_wrap = 3, 
           hue = 'Country', palette = palette)
plt.title('Temperature(°c) vs Clear sky GHI (w/m²)', fontsize = 16)
plt.savefig('Temperature_vs_ClearskyGHI_diff.jpg', dpi= 300)

In [ ]:
# Let's see the GHI trend overtime
plt.figure(figsize= (15,10))
sns.lineplot(x='Date', y='Clear sky GHI', data = country_df, hue= 'Country', palette= palette)
sns.despine()
plt.ylabel('Clear sky GHI(w/m²)', fontsize = 12)
plt.xlabel('Date', fontsize = 12)
plt.title('Clear sky GHI Seasonality across Countries', fontsize = 16)
plt.grid()
#plt.savefig('ClearskyGHI_Seasonality.jpg', dpi= 300)

In [ ]:
plt.figure(figsize= (15,10))
sns.lineplot(x='Date', y='Clear sky GHI', data = country_resamp, hue= 'Country', palette= palette)
sns.scatterplot(x='Date', y='Clear sky GHI', data = country_resamp, hue= 'Country', palette= palette, legend= False)

sns.despine()
plt.ylabel('Clear sky GHI(w/m²)', fontsize = 12)
plt.xlabel('Date', fontsize = 12)
plt.title('Monthly Average Clear sky GHI Seasonality across Countries', fontsize = 16)
plt.grid()
#plt.savefig('Average_ClearskyGHI_Seasonality.jpg', dpi= 300)

In [ ]:
# comparing the seasonality of temperature with the clear sky GHI values overtime across all countries

for na in country_names:
    sub = country_df.copy()
    sub = sub[sub.Country == na][['Date', 'Temperature', 'Clear sky GHI']]
    sub.set_index('Date', inplace= True)
    print('\n', na)
    sub.plot(subplots = True, title = 'Temperature(°c) and Clear sky GHI(w/m²) Seasonalities ' + na, figsize = (12,10))
    #plt.show()
    #plt.savefig('Temp_and_clear_'+na+'.jpg', dpi= 300)

In [ ]:
for na in country_names:
    sub = country_resamp.copy()
    sub.rename({'Temperature':'Average Temperature(°c)', 'Clear sky GHI':'Average Clear sky GHI(w/m²)'}
               ,axis=1, inplace= True)
    sub = sub[sub.Country == na][['Date', 'Average Temperature(°c)', 'Average Clear sky GHI(w/m²)']]
    sub.set_index('Date', inplace= True)
    print('\n', na)
    sub.plot(subplots = True, title = 'Temperature(°c) and Clear sky GHI(w/m²) Monthly Average Seasonalities ' + na, figsize = (12,10),
             marker= '.', fontsize= 16)
    plt.show()
    #plt.savefig('Average_Temp_and_clear_'+na+'.jpg', dpi= 300)

We notice seasonality and that the Highest GHI values occur towards or during Summer for all countries except Brasilia. 

Observations
1. Although for the hottest regions, we noticed a higher distribution range of clear sky GHI, the general temperature value itself doesn't affect the clear sky GHI as we notice that the coldest region, Antartica has the highest clear sky GHI values.
2. GHI value is influenced by season across all countries.

#### Exploring relationship between Wind Speed and Clear Sky GHI

In [ ]:
# Barplot showing wind speed average and confidence interval for the mean across countries

order = country_df.groupby('Country', as_index= False)['Wind speed'].mean().sort_values('Wind speed', ascending= False)

plt.figure(figsize= (12,7))
ax = sns.barplot(x= 'Country', y='Wind speed', data = country_df, palette= palette1, ci= 0, 
            order=order.Country)
ax.bar_label(ax.containers[0])
sns.despine()
plt.xlabel('Country', fontsize = 12)
plt.ylabel('Wind Speed(m/s)', fontsize = 12)
plt.title('Average Wind speed(m/s)', fontsize = 16)
#plt.savefig('Average_Windspeed.jpg', dpi= 300)


In [ ]:
# Let's see the windspeed distribution
plt.figure(figsize= (12,7))
sns.boxplot(x='Country', y='Wind speed', data = country_df, hue= 'Country', palette= palette, order= order.Country)
plt.title('Average Wind speed', fontsize = 16)
plt.grid()
#plt.savefig('Boxplot_windspeed.jpg', dpi= 300)

Observations
1. Antartica is usually wind.
2. Brasillia is almost never windy.

In [ ]:
# Wind speed and clear sky GHI
plt.figure(figsize= (15,10))
sns.scatterplot(x='Wind speed', y='Clear sky GHI', data = country_df, hue= 'Country', palette=palette)
plt.xlabel('Wind speed(m/s)', fontsize= 12)
plt.ylabel('Clear sky GHI (w/m²)', fontsize = 12)
plt.title('Wind speed(m/s) vs Clear sky GHI (w/m²)', fontsize = 16)
#plt.savefig('Wind_vs_clear.jpg', dpi= 300)


Observation
1. There's no linear relationship between Wind speed and clear sky GHI

## Opening and resizing all images

In [ ]:
# Specifying the path to the pictures and listing all picture directories
pic_path = 'Research Paper/EDA-Pictures/'
picture_folders = os.listdir(pic_path)

In [ ]:
for img_dir in picture_folders:
    # creating a 'resized' folder in each of the picture folders
    os.makedirs(pic_path+img_dir+'/resized')

In [ ]:
for img_dir in picture_folders:
    # the img_dir is a name of one picture directory so direc contains all files and folders in each picture directory. I had
    # to remove the 'resized' folder from that list since it wasn't an image
    direc = os.listdir(pic_path+img_dir)
    direc.remove('resized')
    
    for img in direc:
        # I basically just open all pictures, resize and save in the resized folder in each picture folder 
        image = Image.open(pic_path+img_dir+'/'+img)
        print('Resizing '+img+ '...')
        new_image = image.resize((4667, 3500))
        new_image.save(pic_path+img_dir+'/resized/resized_'+img, dpi = (300, 300))
        print('Completed \n')

<a name="loading"></a>
# 5. Power Output
loading data with target

In [ ]:
def clean_data(path):
    # Opening the excel file and arranging
    data = pd.read_excel(path, header = 1)
    data = data.drop(0).iloc[:, :6]
    
    
    # changing column names
    data.rename({'G':'Solar_Irradiance', 'v':'Wind_Speed', 'Ta':'Ambient_Temp'}, axis= 1, inplace= True)
    
    # converting columns to correct data type
    for col in data.columns:
        if col != 'Date':
            data[col] = data[col].astype('float64')
        else:
            data[col] = pd.to_datetime(data[col])
    data.sort_values('Date', inplace= True)
    return data

In [ ]:
# Reading and cleaning data
antarctica_df = clean_data('Train data/Antarctica.xlsx')

In [ ]:
antarctica_df.head()

<a name="power"></a>
## 6. EDA Power Output

In [ ]:
# features for better EDA
antarctica_df['power_bin'] = pd.qcut(antarctica_df['P_PV-TE'], q= [0, 0.4, 0.45, 0.5, 0.57, 0.63, 0.69, 0.76, 0.83, 0.9, 1],
                                     labels = ['lowest', 'lower','low','mediest', 'medium','mid',
                               'high','higher','highest','peak'])
antarctica_df['eff_bin'] = pd.qcut(antarctica_df['N_PV-TE'], q= [0, 0.4, 0.45, 0.5, 0.57, 0.63, 0.69, 0.76, 0.83, 0.9, 1],
                                     labels = ['lowest', 'lower','low','mediest', 'medium','mid',
                               'high','higher','highest','peak'])

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Solar_Irradiance', y='P_PV-TE', data = antarctica_df)

Observation

1.There's an almost perfect linear relationship between the power output and Solar Irradiance.

In [ ]:
# All days where solar irradiance is 0, power should be 0 
antarctica_df[np.logical_and(antarctica_df.Solar_Irradiance ==0, antarctica_df['P_PV-TE'] == 0)]

In [ ]:
# finding the day where the above doesn't happen
antarctica_df[np.logical_and(antarctica_df.Solar_Irradiance ==0, antarctica_df['P_PV-TE'] > 0)]

# this day is actually an outlier and affects both power and efficiency

<a name="outlier_handled" ></a>
#### Outlier_handled

In [ ]:
# I'll be replacing the values of power and efficiency with 0, since that's what it should be.
antarctica_df.iloc[967, 4:6] = [0,0]

In [ ]:
# Confirmation
antarctica_df[np.logical_and(antarctica_df.Solar_Irradiance ==0, antarctica_df['P_PV-TE'] > 0)]

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Ambient_Temp', y='P_PV-TE', data = antarctica_df)

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='P_PV-TE', y='Wind_Speed', data = antarctica_df, alpha = 0.5)

In [ ]:
# average power across bins
antarctica_df.groupby('power_bin')['P_PV-TE'].mean()

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Ambient_Temp', y='Solar_Irradiance', data = antarctica_df, hue= 'power_bin')

<a name = 'above'></a>
#### Binned

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Wind_Speed', y='Solar_Irradiance', data = antarctica_df, hue= 'power_bin')

In [ ]:
# Comparing Seasonality of Irradiance to the target Variable

sub = antarctica_df.copy()
sub = sub[['Date', 'Solar_Irradiance', 'P_PV-TE']]
sub.set_index('Date', inplace= True)
sub.plot(subplots = True, title = 'Solar Irradiance(w/m²) and P', figsize = (12,10))


Observations
1. There's very similar seasonality between solar Irradiance and the target variable and both are stationary that explains the almost perfect linear relationship.

In [ ]:
# Comparing Seasonality of Temperature to the target Variable

sub = antarctica_df.copy()
sub = sub[['Date', 'Ambient_Temp', 'P_PV-TE']]
sub.set_index('Date', inplace= True)
sub.plot(subplots = True, title = 'Temperature and P', figsize = (12,10))

In [ ]:
# Comparing trend of Wind Speed to the target Variable

sub = antarctica_df.copy()
sub = sub[['Date', 'Wind_Speed', 'P_PV-TE']]
sub.set_index('Date', inplace= True)
sub.plot(subplots = True, title = 'Wind_Speed and P', figsize = (12,10))


<a name="Engineering"></a>
## 7. Feature Creation

The target Variable has a trend with time there's basically a polynomial relationship between day of year and the target variable.

In [ ]:
antarctica_df['month'] = antarctica_df['Date'].dt.month
antarctica_df['dayofyear'] = antarctica_df['Date'].dt.dayofyear
antarctica_df['day'] = antarctica_df['Date'].dt.day
antarctica_df['weekday'] = antarctica_df['Date'].dt.weekday
antarctica_df['weekofyear'] = antarctica_df['Date'].dt.weekofyear
#antarctica_df['year'] = antarctica_df['Date'].dt.year


In [ ]:
antarctica_df.head()

In [ ]:
# Plotting all time features using a loop
for time_feat in list(antarctica_df.columns)[8:]:
    plt.figure(figsize= (12,7))
    sns.scatterplot(x=time_feat, y='P_PV-TE', data = antarctica_df)

Observations

We notice that there's mostly polynomial relationship between the time features and the power output 

In [ ]:
# Correlation Heatmap 
corr = antarctica_df.corr()
plt.figure(figsize = (13, 8))
sns.heatmap(corr, cmap='RdYlGn', annot = True, center = 0)
plt.title('Correlogram', fontsize = 15, color = 'darkgreen')
plt.show()

Observations
1. We notice a perfect positive correlation between Solar Irradiance and P_PV-TE, we also notice a high positive correlation with ambient temp and a low negative correlation with windspeed.

In [ ]:
def scale_datasets(X_train, X_test, kind= 'standard'):
    '''Performs minmax or standard scaling on thrain and test datasets'''
    if kind == 'standard':
        standard_scaler = StandardScaler()
        x_train_scaled = pd.DataFrame(
          standard_scaler.fit_transform(X_train),
          columns=X_train.columns
      )
        x_test_scaled = pd.DataFrame(
          standard_scaler.transform(X_test),
          columns = X_test.columns
      )
    elif kind == 'min':
        mm_scaler = MinMaxScaler()
        x_train_scaled = pd.DataFrame(
          mm_scaler.fit_transform(X_train),
          columns=X_train.columns
      )
        x_test_scaled = pd.DataFrame(
          mm_scaler.transform(X_test),
          columns = X_test.columns
      )
    return x_train_scaled, x_test_scaled

In [ ]:
def plot_feat_imp(train_data, model):
    '''Plots the feature importances of models'''
    plt.bar(train_data.columns, model.feature_importances_)
    plt.title('Feature Importances for'+model.__class__.__name__)
    plt.xticks(rotation = 90)
    plt.show()
    
def plot_real_pred(X_test, y_test, model):
    '''Plots the real values against the predicted values of any model'''
    plt.figure(figsize = (10,6))
    plt.scatter(y_test, model.predict(X_test))
    plt.xlabel('Real Values')
    plt.ylabel('Predicted Values')
    plt.grid()

<a name="Final"></a>
## 8. Spliting data as time Series
Using 2021 as test data.

In [ ]:
n_train = antarctica_df.shape[0] - round(antarctica_df.shape[0]*0.333)
n_train

In [ ]:
X_traina = antarctica_df.drop(['P_PV-TE', 'N_PV-TE', 'power_bin', 'eff_bin'], axis= 1).iloc[:n_train, :]
y_traina = antarctica_df[['P_PV-TE']].iloc[:n_train, :]

X_testa = antarctica_df.drop(['P_PV-TE', 'N_PV-TE', 'power_bin', 'eff_bin'], axis= 1).iloc[n_train:, :]
y_testa = antarctica_df[['P_PV-TE']].iloc[n_train:, :]

In [ ]:
X_training = X_traina.drop('Date', axis = 1)
X_testing = X_testa.drop('Date', axis = 1)

In [ ]:
# Earliest day in the test set is January 1, 2021
X_testa.Date.min()

<a name="sensitivity_analysis"></a>
## 9. Sensitivity Analysis
Global Sensitivity Analysis Using Linear Regression. From the EDA, temperature and solar Irradiance are linearly related to 
the target but windspeed doesn't seem to have any relationship although there's a medium -ve correlation with windspeed and power output.

In [ ]:
lin_reg = LinearRegression()

In [ ]:
# Scaling features so the coefficient magnitude makes sense.
X_training, X_testing = scale_datasets(X_training, X_testing, kind ='min')

In [ ]:
X_training

In [ ]:
lin_reg.fit(X_training, y_traina)

In [ ]:
pred = lin_reg.predict(X_testing)

In [ ]:
plot_real_pred(X_testing, y_testa, lin_reg)

In [ ]:
## getting the coefficients of the linear regression model
lin_reg.coef_

In [ ]:
plt.bar(X_testing.columns, lin_reg.coef_[0])
plt.xticks(rotation = 90)

In [ ]:
np.sqrt(mean_squared_error(y_testa, pred))

**Observation.**

From the coefficients of the Linear regression model, we can see that the predictions are ruled by Solar Irradiance with the highest coefficient, day of year and then month with a negative correlation. 

However, this results can't be accepted because from the EDA month and day of year aren't libearly related to the power output and as pointed out earlier this is a drawback of using Linear regression for sensitivity analysis.

**To do an acceptable feature selection, Recursive feature elimination was carried out below.**

In [ ]:
## Sensitivity analysis plot showing how power changes with Irradiance and windspeed
antarctica_df['Temperature'] = pd.qcut(antarctica_df['P_PV-TE'], q= [0, 0.40, 0.5, 0.75, 1], labels = ['lowest','low', 'medium', 'high'])

In [ ]:
plt.figure(figsize= (12,8))
sns.scatterplot(x='P_PV-TE', y='Solar_Irradiance', size = 'Wind_Speed', data = antarctica_df, hue= 'Temperature', 
                sizes = (5, 200), alpha= 0.7)
plt.yticks(list(range(0, 1500, 50)))
plt.title('Sensitivity Analysis Plot with Three Features and the Target Variable', fontsize= 15)
plt.ylabel('Solar Irradiance (W/m²)', fontsize = 13)
plt.xlabel('Power (W)', fontsize = 13)
plt.xlim(left = 20, right= 30)
plt.grid()
#plt.savefig('sensitivity_analysis_zoomed.jpg', dpi= 300)

### Brief Introduction and Observations

Until quite recently, sensitivity analysis was conceived and often defined as a local measure of the effect of a given input on a given output. This is customarily obtained by computing via a direct or indirect approach, system derivatives such as Sj = ∂Y/∂ Xj , where Y is the output of interest and Xj an input factor.


Sensitivity analysis can be said to be ‘The study of how the uncertainty in the output of a model (numerical or otherwise) can be apportioned to different sources of uncertainty in the model input’. 

In this plot, we see that as the power Increases, the Solar Irradiance and Temperature Increases. Now, while holding the Solar Irradiance constant we notice that as the windspeed decreases, the power Increases.

<a name="rfecv"></a>
### 10. Recursive Feature Elimination Cross Validation

In [ ]:
# Experimenting on 7 models
regressors = {
    'LGBM': LGBMRegressor(random_state=1),
    'Linear_regression':LinearRegression(),
    'Ridge': Ridge(),
    'XGB': XGBRegressor(random_state = 66),
    'cat': CatBoostRegressor(random_state = 77),
    'RandomForest': RandomForestRegressor(random_state = 99),
    'Decision Tree': DecisionTreeRegressor(random_state = 12),
    'Adaboost': AdaBoostRegressor(random_state= 40)
}

In [ ]:
def recurse(X_train, y_train, model):
    rfecv = RFECV(estimator= model, step=1, scoring = 'neg_mean_squared_error', cv= 5, verbose= 5, n_jobs=-1)
    
    # fitting with data
    rfecv.fit(X_train, y_train)
    
    # reducing the dataset and leaving the n best columns
    # rfecv.transform(X_train)
    
    print('Optimal Number of features for {} is: {}'.format(model.__class__.__name__, rfecv.n_features_))
    print(rfecv.get_feature_names_out())
    
    # prints out the mean cv score when n features were used
    print(rfecv.cv_results_['mean_test_score'], '\n')

In [ ]:
#rfecv = RFECV(estimator= DecisionTreeRegressor(), step=1, scoring = 'neg_mean_squared_error', cv= 5, verbose= 5, n_jobs=-1)
#rfecv.fit(X_training, y_traina)
#print('Optimal Number of features {}'.format(rfecv.n_features_))

#rfecv.transform(X_training)

In [ ]:
for mods in regressors:
    recurse(X_training, y_traina, regressors[mods])

In [ ]:
# manually creating something similar for SVR and KNN since svr in the rbf kernel doesnt provide coef_ or 
# feature importances

models = {'SupportVector':SVR(), 'KNN': KNeighborsRegressor(), 
         'LGBM': LGBMRegressor(random_state=1),
    'XGB': XGBRegressor(random_state = 66)
}

columns = X_testing.columns
scores = []
cols = []
test_scores = []

for model in models:
    print('FOR MODEL '+model)
    for switch in range(len(columns)):
        if switch>=0 and switch<=len(columns)-1:
            current_cols = list(columns)[:switch+1]
            #print('training on columns', current_cols)
            cross_val = cross_val_score(models[model], X_training[current_cols], y_traina, scoring = 'neg_mean_squared_error',
                                                      cv= 5, verbose= False, n_jobs=-1)
            #print('Using {} mean rmse is {}'.format(current_cols, np.mean(cross_val*-1)))
            models[model].fit(X_training[current_cols], y_traina)
            pred = models[model].predict(X_testing[current_cols])
            test_scores.append(np.sqrt(mean_squared_error(y_testa, pred)))
            scores.append(np.mean(cross_val*-1))
            cols.append(current_cols)
            
        else:
            break
    print('For model, we had the lowest crossval score of {} with {}'.format(min(scores), cols[scores.index(min(scores))]))
    print('For model, we had the lowest test score of {} with {}'.format(min(test_scores), 
                                                                         cols[test_scores.index(min(test_scores))]))
    #print('trained on columns', cols, scores)
    print('\n')
    scores = []
    cols = []
    test_scores = []

## Summary. 

From the results obtained above, I'll be modelling with just Solar Irradiance and windspeed.

<a name="desciptionofnewmetrics"></a>
# 11. Description of new metrics
rrmse = $\sqrt{\frac{\sum \limits _{i=1} ^{n} (Y_{i} - \hat{Y}_{i})^{2}}{\sum \limits _{i=1} ^{n} (Y_{i} - \bar{Y})^{2}}}$ x 100

Relative rootmean square error basically compares the root mean sqaure error of the actual values to their predictions against the actual values to the mean of the actual values. Basically for a good model, the error in the former should be a lot smaller than the error in the later i.e rmse wrt mean, hence the fraction will produce a small result. When the model is bad the the rrmse will produce a score close to 1 or greater than 1 multiplying by 100 shows us the value in percentages. The rrmse is a dimensionless metric.

mape = ${\frac{1}{n}\sum \limits _{i=1} ^{n}}\left\lvert{\frac{Y_{i}-\hat{Y_{i}}}{Y_{i}}}\right\rvert$ x 100

The mean absolute percentage error (MAPE) is the mean or average of the absolute percentage errors of forecasts. Error is defined as actual or observed value minus the forecasted value. 
(I don't think this metric is very relevant)

mbe = $\frac{1}{n}\sum \limits _{i=1} ^{n} Y_{i}-\hat{Y}_{i}$

The Mean Bias Error is usually not used as a measure of the model error as high individual errors in prediction can also produce a low MBE. Mean bias error is primarily used to estimate the average bias in the model and to decide if any steps need to be taken to correct the model bias. Mean Bias Error (MBE) captures the average bias in the prediction. A positive bias or error in a variable (such as wind speed) represents the data from datasets is overestimated and vice versa, whereas for the variable direction (such as wind direction) a positive bias represents a clockwise deviation and vice versa. The lower values of errors and considerably higher value of correlation coefficient for the variable and direction are of greater importance.

mabe = It's the same as mean absolute error

I ignored t-stat as I think the current metrics are enough.

<a name="Modelling"></a>
# 12. Modelling

In [ ]:
# Models to try
regressors = {
    'LGBM': LGBMRegressor(random_state=1),
    'Linear_regression':LinearRegression(),
    'Ridge': Ridge(),
    'XGB': XGBRegressor(random_state = 66),
    'cat': CatBoostRegressor(random_state = 77),
    'RandomForest': RandomForestRegressor(random_state = 99),
    'Decision Tree': DecisionTreeRegressor(random_state = 12),
    'Adaboost': AdaBoostRegressor(random_state= 40),
    'Gboosting': GradientBoostingRegressor(random_state= 42),
    'Bagging': BaggingRegressor(random_state= 42), 
    'SVR':SVR(),
    'KNN':KNeighborsRegressor()
}

# Reporting function
def train_report(X_train, X_test, y_train, y_test):
    df_models = pd.DataFrame(columns=['model', 'run_time', 'rmse', 'rmse_cv', 'mae', 'r2', 'mape', 'rrmse', 'mbe'])

    for key in regressors:

        start_time = time.time()

        regressor = regressors[key]
        model = regressor.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        # (1/len(y_pred)*np.sum([abs((a-b)/a) for a, b in zip(y_test.values, y_pred)]))*100
        rrmse = np.sqrt((mean_squared_error(y_test, y_pred)/mean_squared_error(y_test, [np.mean(y_test)]*len(y_test))))*100
        mbe = 1/len(y_pred)*np.sum(y_test.values-y_pred)

        scores = cross_val_score(model, 
                             X_train, 
                             y_train,
                             scoring="neg_mean_squared_error", 
                             cv=5, verbose = 3, n_jobs = -1)

        row = {'model': key,
           'run_time': format(round((time.time() - start_time)/60,2)),
           'rmse': round(rmse, 3),
           'rmse_cv': round(np.mean(np.sqrt(-scores)), 3),
           'mae': round(mae, 3),
            'r2': round(r2, 3),
            'mape': mape, 
            'rrmse': rrmse,
               'mbe':mbe
        }

        df_models = df_models.append(row, ignore_index=True)
        
    display(df_models)  
    
    # dispaying the top 5 models wrt rmse
    display(df_models.sort_values('rmse').iloc[:7,:].reset_index(drop = True))
    

In [ ]:
# Using just two Features
X_train = X_training[['Solar_Irradiance', 'Wind_Speed']]

X_test = X_testing[['Solar_Irradiance', 'Wind_Speed']]

In [ ]:
train_report(X_train, X_test, y_traina, y_testa)

<a name="Tuning"></a>
# 13. Model Tuning
Tuning the Top 5 models except catboost, plus SVM, KNN and ANN.

In [ ]:
def compare_tuned(model_map, X_train, X_test, y_train, y_test):
    df_models = pd.DataFrame(columns=['model', 'run_time', 'rmse', 'mae', 'r2', 'mape'])

    for a, key in enumerate(model_map):

        start_time = time.time()

        regressor = model_map[key]
        model = regressor.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        rrmse = np.sqrt((mean_squared_error(y_test, y_pred)/mean_squared_error(y_test, [np.mean(y_test)]*len(y_test))))*100

        
        row = {'model': key,
           'run_time': format(round((time.time() - start_time)/60,2)),
           'rmse': round(rmse, 3),
            'mae':round(mae, 3),
            'r2': round(r2, 3),
            'mape': round(mape, 3), 
            'RRMSE': rrmse
        }

        df_models = df_models.append(row, ignore_index=True)
    
    # dispaying the top 5 models wrt rmse
    display(df_models.sort_values('rmse').iloc[:,:].reset_index(drop = True))

## 1. LGBMRegressor


I actually tuned a lot of Values for each parameter but I narrowed it down to this.

In [ ]:
model_seed= 42

In [ ]:
lgbm_params = {'boosting_type':['dart'], 'max_depth':[-1], 'num_leaves':[3], 
               'learning_rate':[0.3], 'n_estimators': [7000], 'reg_lambda':[0.4],
               'reg_alpha':[0.45], 'colsample_bytree': [0.9], 'min_child_samples':[1]}

In [ ]:
grid_lgb = GridSearchCV(LGBMRegressor(random_state = model_seed), param_grid = lgbm_params, 
                        scoring= 'neg_root_mean_squared_error', cv= 5, n_jobs = -1, 
                       verbose= 10)

In [ ]:
%%time
grid_lgb.fit(X_train, y_traina)

In [ ]:
grid_lgb.best_params_

In [ ]:
grid_lgb.best_score_

### Tuning History

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': -1,
 'min_child_samples': 1,
 'n_estimators': 7000,
 'num_leaves': 3,
 'reg_alpha': 0.45,
 'reg_lambda': 0.4}


-0.13176514876298723

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': -1,
 'min_child_samples': 1,
 'n_estimators': 7000,
 'num_leaves': 3,
 'reg_alpha': 0.4,
 'reg_lambda': 0.5}

-0.13194496550772286

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': -1,
 'min_child_samples': 2,
 'n_estimators': 6700,
 'num_leaves': 3,
 'reg_alpha': 0.4,
 'reg_lambda': 1}

-0.13286796072481544

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': -1,
 'min_child_samples': 1,
 'n_estimators': 3500,
 'num_leaves': 3,
 'reg_alpha': 0.5,
 'reg_lambda': 1}

-0.1336833356493271

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': -1,
 'min_child_samples': 1,
 'n_estimators': 3000,
 'num_leaves': 3,
 'reg_alpha': 0.9,
 'reg_lambda': 1.2}

-0.14230706031253967

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': -1,
 'min_child_samples': 5,
 'n_estimators': 2500,
 'num_leaves': 3,
 'reg_alpha': 1,
 'reg_lambda': 1}
-0.14230706031253967

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.4,
 'max_depth': -1,
 'n_estimators': 2000,
 'num_leaves': 3,
 'reg_alpha': 0.5,
 'reg_lambda': 1}

-0.17973896563125008

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': -1,
 'n_estimators': 1400,
 'num_leaves': 3,
 'reg_alpha': 1,
 'reg_lambda': 0.5}
-0.18556084101286693

### Getting Final results

Gridsearch retrains the best model on the whole dataset as seen. You can control this behaviour with the boolean refit parameter (True by default).

In [ ]:
best_lgbm = grid_lgb.best_estimator_

In [ ]:
lgb_map = {'Untuned model':LGBMRegressor(), 'Tuned': best_lgbm}

In [ ]:
compare_tuned(lgb_map, X_train, X_test, y_traina, y_testa)

We notice a massive improvement on the tuned LGBM on the test data wrt the metrics.

In [ ]:
plot_feat_imp(X_train, best_lgbm)

In [ ]:
plot_real_pred(X_test, y_testa, best_lgbm)

## 2. Gradient Boosting Regressor

In [ ]:
grad_params = {'n_estimators':[3000, 5000], 'criterion':['squared_error'], 'min_samples_split':[2], 
              'min_samples_leaf':[1], 'max_depth': [3], 'max_leaf_nodes': [10],
               'learning_rate':[0.05]}

In [ ]:
grid_grad = GridSearchCV(GradientBoostingRegressor(random_state = model_seed), param_grid = grad_params, 
                          scoring= 'neg_root_mean_squared_error', cv= 5,
                       n_jobs = -1, verbose= 10)

In [ ]:
%%time
grid_grad.fit(X_train, y_traina)

In [ ]:
grid_grad.best_params_

In [ ]:
grid_grad.best_score_

### Tuning History

{'criterion': 'squared_error',
 'learning_rate': 0.05,
 'max_depth': 3,
 'max_leaf_nodes': 50,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 3000}

-0.19161785497276798

{'criterion': 'squared_error',
 'learning_rate': 0.05,
 'max_depth': 3,
 'max_leaf_nodes': 300,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 2000}

-0.19192767358160737

{'criterion': 'squared_error',
 'learning_rate': 0.05,
 'max_depth': 3,
 'max_leaf_nodes': 1000,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1500}

-0.19298592040776325

### Getting Final results

In [ ]:
best_grad = grid_grad.best_estimator_

In [ ]:
grad_map = {'Untuned model':GradientBoostingRegressor(), 'Tuned': best_grad}

In [ ]:
compare_tuned(grad_map, X_train, X_test, y_traina, y_testa)

In [ ]:
plot_feat_imp(X_train, best_grad)

In [ ]:
plot_real_pred(X_test, y_testa, best_grad)

## 3. XGB Regressor

In [ ]:
xgb_params = {'n_estimators':[1000, 1500], 'booster':['gbtree'], 'max_depth':[2,3], 
              'max_leaves':[0], 'colsample_bynode': [1],
               'learning_rate':[0.3], 'reg_lambda':[0.6, 1], 'reg_alpha':[0, 0.05, 0.1],
               'colsample_bytree': [1]}

In [ ]:
grid_xgb = GridSearchCV(XGBRegressor(random_state = model_seed), param_grid = xgb_params, 
                        scoring= 'neg_root_mean_squared_error', cv= 5, n_jobs = -1, 
                       verbose= 10)

In [ ]:
%%time
grid_xgb.fit(X_train, y_traina)

In [ ]:
grid_xgb.best_params_ 

In [ ]:
grid_xgb.best_score_

### Tuning History

{'booster': 'gbtree',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'learning_rate': 0.3,
 'max_depth': 2,
 'max_leaves': 0,
 'n_estimators': 1000,
 'reg_alpha': 0.05,
 'reg_lambda': 0.6}

-0.18283240962275818

{'booster': 'gbtree',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 2,
 'max_leaves': 0,
 'n_estimators': 5000,
 'reg_alpha': 0,
 'reg_lambda': 0.6}

-0.1864575567179877

{'booster': 'gbtree',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 2,
 'max_leaves': 0,
 'n_estimators': 3000,
 'reg_alpha': 0,
 'reg_lambda': 0.6}

-0.1879322273234872

{'booster': 'gbtree',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 2,
 'max_leaves': 0,
 'n_estimators': 2000,
 'reg_alpha': 0,
 'reg_lambda': 0.6}

-0.19017287374753922

{'booster': 'dart',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 3,
 'max_leaves': 0,
 'n_estimators': 1000,
 'reg_alpha': 0,
 'reg_lambda': 0.4}

-0.19977789651466304

{'booster': 'gbtree',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 4,
 'max_leaves': 0,
 'n_estimators': 400,
 'reg_alpha': 0,
 'reg_lambda': 0.3}

 -0.21065251367419258

### Getting Final results

In [ ]:
best_xgb = grid_xgb.best_estimator_

In [ ]:
xgb_map = {'Untuned model':XGBRegressor(), 'Tuned': best_xgb}

In [ ]:
compare_tuned(xgb_map, X_train, X_test, y_traina, y_testa)

In [ ]:
plot_feat_imp(X_train, best_xgb)

In [ ]:
plot_real_pred(X_test, y_testa, best_xgb)

## 4. Random Forest Regressor

In [ ]:
rf_params = {'n_estimators':[300,500], 'max_depth':[None], 'min_samples_split': [2,3],
             'min_samples_leaf': [1,2], 'max_leaf_nodes':[None, 300]}

In [ ]:
grid_rf = GridSearchCV(RandomForestRegressor(random_state = model_seed ), param_grid = rf_params, 
                       scoring= 'neg_root_mean_squared_error', cv= 5,
                       n_jobs = -1, verbose= 10)

In [ ]:
%%time
grid_rf.fit(X_train, y_traina)

In [ ]:
grid_rf.best_params_

In [ ]:
grid_rf.best_score_

### Tuning History

{'max_depth': None,
 'max_leaf_nodes': 500,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 500}

-0.33271110775670376

{'max_depth': None,
 'max_leaf_nodes': 500,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 800}

-0.3744656348191273

### Getting Final results

In [ ]:
best_rf = grid_rf.best_estimator_

In [ ]:
rf_map = {'Untuned model':RandomForestRegressor(), 'Tuned': best_rf}

In [ ]:
compare_tuned(rf_map, X_train, X_test, y_traina, y_testa)

There's no visible improvement with Random Forest

In [ ]:
plot_feat_imp(X_train, best_rf)

In [ ]:
plot_real_pred(X_test, y_testa, best_rf)

## 5. Bagging Regressor

In [ ]:
bag_params = {'n_estimators':[190, 200, 250],'max_samples':[ 0.8, 0.9, 1],
               'warm_start': [False], 'max_features':[1,2]}

In [ ]:
grid_bag = GridSearchCV(BaggingRegressor(random_state = model_seed), param_grid = bag_params,
                         scoring= 'neg_root_mean_squared_error', cv= 5,
                       n_jobs = -1, verbose= 10)

In [ ]:
%%time
grid_bag.fit(X_train, y_traina)

In [ ]:
grid_bag.best_params_

In [ ]:
grid_bag.best_score_

### Tuning History

{'max_features': 2,
 'max_samples': 0.8,
 'n_estimators': 200,
 'warm_start': False}

-0.345392476322652

{'max_features': 2,
 'max_samples': 0.7,
 'n_estimators': 200,
 'warm_start': False}

-0.35682548954107823

### Getting Final results

In [ ]:
best_bag = grid_bag.best_estimator_

In [ ]:
bag_map = {'Untuned model':BaggingRegressor(), 'Tuned': best_bag}

In [ ]:
compare_tuned(bag_map, X_train, X_test, y_traina, y_testa)

In [ ]:
plot_real_pred(X_test, y_testa, best_bag)

## 6. Support Vector Machine
I think the SVM has a good potential especially when other kernels are tried although in this case a linear kernel isn't bad as the main feature has a good linear relationship with the target.

In [ ]:
x_train_scaledm, x_test_scaledm = scale_datasets(X_training, X_testing, kind ='min')
x_train_scaled, x_test_scaled = scale_datasets(X_training, X_testing)

In [ ]:
X_testing

In [ ]:
sv = SVR()

In [ ]:
sv.fit(x_train_scaledm[['Solar_Irradiance', 'Wind_Speed']], y_traina)

In [ ]:
np.sqrt(mean_squared_error(y_testa, sv.predict(x_test_scaledm[['Solar_Irradiance', 'Wind_Speed']])))

In [ ]:
sv = SVR()

In [ ]:
sv.fit(x_train_scaled[['Solar_Irradiance', 'Wind_Speed']], y_traina)

In [ ]:
np.sqrt(mean_squared_error(y_testa, sv.predict(x_test_scaled[['Solar_Irradiance', 'Wind_Speed']])))

### MinMax scaler performed better than Standard Scaler for SVM

In [ ]:
svr_params = {'kernel':['rbf'], 'degree':[1,2], 'C':[900,1000, 1500, 2000, 3000, 5000], 
              'gamma':[1.2,1.25, 1.3, 1.4]} 
              

In [ ]:
grid_svr = GridSearchCV(SVR(), param_grid = svr_params, scoring= 'neg_root_mean_squared_error', cv= 5, n_jobs = -1, 
                       verbose= 10)

In [ ]:
%%time
grid_svr.fit(X_train, y_traina)

In [ ]:
grid_svr.best_params_ 

In [ ]:
grid_svr.best_score_

### Tuning History

{'C': 1000, 'degree': 1, 'gamma': 1.25, 'kernel': 'rbf'}
-0.06370991670598065

{'C': 500, 'degree': 1, 'gamma': 1.25, 'kernel': 'rbf'}
-0.06732201416291216

### Getting Final results

In [ ]:
best_svr = grid_svr.best_estimator_

In [ ]:
svr_map = {'Untuned model':SVR(), 'Tuned': best_svr}

In [ ]:
compare_tuned(svr_map, X_train, X_test, y_traina, y_testa)

In [ ]:
plot_real_pred(X_test, y_testa, best_svr)

# 7. KNN Regressor

In [ ]:
knn_params = {'n_neighbors':[2,3,4,5,6,7,8,9,10,11,12,13], 'weights':['uniform', 'distance'], 'p':[1,2,3], 
             'metric': ['euclidean', 'manhattan', 'minkowski']} 
              

In [ ]:
grid_knn = GridSearchCV(KNeighborsRegressor(), param_grid = knn_params, scoring= 'neg_root_mean_squared_error', cv= 5, n_jobs = -1, 
                       verbose= 10)

In [ ]:
%%time
grid_knn.fit(X_train, y_traina)

In [ ]:
grid_knn.best_params_ 

In [ ]:
grid_knn.best_score_

### Getting Final results

In [ ]:
best_knn = grid_knn.best_estimator_

In [ ]:
knn_map = {'Untuned model':KNeighborsRegressor(), 'Tuned': best_knn}

In [ ]:
compare_tuned(knn_map, X_train, X_test, y_traina, y_testa)

In [ ]:
plot_real_pred(X_test, y_testa, best_knn)

# 8. Artificial Neural Network (ANN)

In [ ]:
hidden_units1 = 384
hidden_units2 = 32
hidden_units3 = 64
hidden_units4 = 96
hidden_units5 = 64
learning_rate = 0.009
# Creating model using the Sequential in tensorflow
def build_model_using_sequential():
    model = Sequential([
    tf.keras.Input(shape=(2,)),
    Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
    Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
    Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
    Dense(hidden_units4, kernel_initializer='normal', activation='relu'),
    Dense(hidden_units5, kernel_initializer='normal', activation='relu'),

    Dense(1, kernel_initializer='normal', activation='linear')
  ])
    return model

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15)

In [ ]:
x_train_scaled, x_test_scaled = x_train_scaled[['Solar_Irradiance', 'Wind_Speed']], x_test_scaled[['Solar_Irradiance', 'Wind_Speed']]

In [ ]:
mse = MeanSquaredError()
model = build_model_using_sequential()
model.compile(
    loss=mse, 
    optimizer=Adam(learning_rate=learning_rate), 
    metrics=[mse]
)
# train the model
history = model.fit(
    x_train_scaled, 
    y_traina, 
    epochs=50, 
    batch_size=32,
    validation_split=0.2, callbacks = [early_stopping]
)

In [ ]:
def plot_history(history, key):
    plt.plot(history.history[key])
    plt.plot(history.history['val_'+key])
    plt.xlabel("Epochs")
    plt.ylabel(key)
    plt.legend([key, 'val_'+key])
    plt.show()
    
# Plotting the history
plot_history(history, 'mean_squared_error')

In [ ]:
annpr = model.predict(x_test_scaled)

In [ ]:
np.sqrt(mean_squared_error(y_testa, annpr))

## Tuning the Neural Network using Keras Tuner

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  tf.keras.Input(shape=(2,))

  
  for i in range(hp.Int('layer', 2, 15)):
    model.add(Dense(hp.Int('units_'+ str(i), min_value=32, max_value=512, step=32), kernel_initializer='normal',
                    activation='relu'))  

  hp_learning_rate = hp.Choice('learning_rate', values=[0.0001, 0.001, 0.005])
  
  model.add(Dense(1, kernel_initializer='normal', activation='linear'))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.MeanSquaredError(),
                metrics=['mse'])

  return model

In [ ]:
# max trials just says how many combination of hyperparameters should I try, them executions say for each combination, how
# many times should I train on that ? Because for NNs we get different results for the same model

tuner = kt.RandomSearch(model_builder,
                     objective=kt.Objective("val_mse", direction="min"),
                     max_trials=50,
                     executions_per_trial=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

In [ ]:
tuner.search(x_train_scaled.values, y_traina.values, epochs=50, validation_split=0.2, callbacks=[stop_early],
             batch_size =64)

# Getting the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
tuner.results_summary()

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# Building the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train_scaled, y_traina, epochs=50, validation_split=0.2, batch_size =64)

val_acc_per_epoch = history.history['val_mse']
best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retraining the model
hypermodel.fit(x_train_scaled, y_traina, epochs=best_epoch, validation_split=0.2, batch_size = 64)

In [ ]:
eval_result = hypermodel.evaluate(x_test_scaled, y_testa)
print("[test loss, test mse]:", eval_result)

In [ ]:
hypermodel.summary()

In [ ]:
#hypermodel.save('best_power.h5')

## I got the best result after tuning and saved it. That's what was used below.

<a name="summary_power"></a>
## 14. Summary and Conclusions for Efficiency

In [ ]:
def visualize_pred(X_train, X_test, y_train, y_test, train_pr, test_pr, model):
    '''Plots the predicted and real values overtime'''
    plt.figure(figsize = (12,8))
    plt.plot(X_train.Date, y_train, color = 'b', label = 'Real')
    plt.plot(X_train.Date, train_pr, color = 'r', label = model)
    plt.text(pd.to_datetime('04/25/2019'),30, 'Training Data', fontsize = 14)
    plt.text(pd.to_datetime('04/25/2021'),30, 'Testing Data', fontsize = 14)

    plt.plot(X_test.Date, y_test, color = 'b')
    plt.plot(X_test.Date, test_pr, color = 'r')
    plt.axvline(X_test.Date.min(), color= 'k', ls = '--')
    plt.xlabel('Date', fontsize = 14)
    plt.ylabel('Power Output (W)', fontsize = 14)
    
    plt.legend()
    #plt.savefig('pred'+model+'.jpg', dpi = 300)
    plt.show()
    

In [ ]:
def total_report(model_map, X_traindate, X_testdate, X_train, X_test, X_trainsc, X_testsc, y_train, y_test ):
    df_models = pd.DataFrame(columns=['model', 'Time', 'MSE', 'RMSE', 'MAE', 'r2', 'mape', 'RRMSE', 'Error'])

    for a, key in enumerate(model_map):
        if key == 'ANN':
            regressor = model_map[key]
            start_time = time.time()
            regressor.fit(X_trainsc, y_train, epochs=50, batch_size= 32)
            end_time = time.time()
            y_pred_tr = regressor.predict(X_trainsc)
            start_time_pr = time.time()
            y_pred_te = regressor.predict(X_testsc)
            end_time_pr = time.time()
        else:
            regressor = model_map[key]
            start_time = time.time()
            regressor.fit(X_train, y_train)
            end_time = time.time()
            y_pred_tr = regressor.predict(X_train)
            start_time_pr = time.time()
            y_pred_te = regressor.predict(X_test)
            end_time_pr = time.time()
            
        # metrics
        train_time = end_time - start_time
        pr_time = end_time_pr - start_time_pr
        #print(pr_time, train_time)
        #tr_te = ['train', 'test']
        rmse = [np.sqrt(mean_squared_error(y_train, y_pred_tr)),  np.sqrt(mean_squared_error(y_test, y_pred_te))] 
        mse = [mean_squared_error(y_train, y_pred_tr), mean_squared_error(y_test, y_pred_te)]
        mae = [mean_absolute_error(y_train, y_pred_tr), mean_absolute_error(y_test, y_pred_te)]
        r2 = [r2_score(y_train, y_pred_tr), r2_score(y_test, y_pred_te)]
        mape = [mean_absolute_percentage_error(y_train, y_pred_tr), mean_absolute_percentage_error(y_test, y_pred_te)]
        rrmse = [np.sqrt((mean_squared_error(y_train, y_pred_tr)/mean_squared_error(y_train, 
                                                                                    [np.mean(y_train)]*len(y_train))))*100,
                 np.sqrt((mean_squared_error(y_test, y_pred_te)/mean_squared_error(y_test, [np.mean(y_test)]*len(y_test))))*100]

        
        row_tr = {'model': key,
           'Time_trte': train_time,
            'MSE':mse[0],
           'RMSE': rmse[0],
            'MAE':mae[0],
            'r2': r2[0],
            'mape': mape[0] ,
            'RRMSE': rrmse[0],
            'Error': 'Train',
            'Time': 'Training'
        }
                      
        row_te = {'model': key,
           'Time_trte': pr_time,
            'MSE':mse[1],
           'RMSE': rmse[1],
            'MAE':mae[1],
            'r2': r2[1],
            'mape': mape[1] ,
            'RRMSE': rrmse[1],
            'Error': 'Test',
             'Time': 'Testing'
        }
        visualize_pred(X_traindate, X_testdate, y_train, y_test, y_pred_tr, y_pred_te, key)
        df_models = df_models.append(row_tr, ignore_index=True)
        df_models = df_models.append(row_te, ignore_index=True)
                                                                    
    # dispaying the top 5 models wrt rmse
    display(df_models)
    sorted_df = df_models.sort_values(['RMSE', 'Error'], ascending = False).iloc[:,:].reset_index(drop = True)
    sorted_df = round(sorted_df, 3)
    display(sorted_df)
    display_metrics = ['MSE', 'RMSE', 'MAE', 'RRMSE', 'Time_trte']
    for metric in display_metrics:
        if metric == 'Time_trte':
            palette = {'Training':'red', 'Testing':'green'}
            sorted_df = df_models.sort_values(['Time_trte', 'Time'], ascending = False).iloc[:,:].reset_index(drop = True)
            sorted_df = round(sorted_df, 3)
            plt.figure(figsize = (12,8))
            ax0 = sns.barplot(x= 'model', y=metric, data= sorted_df, hue= 'Time', palette= palette ) 
            plt.ylabel('Time (s)', fontsize = 13)
            plt.xlabel('Model', fontsize = 13)
            plt.title('Training and Testing Time for Eight Supervised Learning Models', fontsize = 15)
            for container in ax0.containers:
                ax0.bar_label(container)
            #plt.savefig('metric'+metric+'.jpg', dpi = 300)
            plt.show()
        else:
            plt.figure(figsize = (12,8))
            ax0 = sns.barplot(x= 'model', y=metric, data= sorted_df, hue= 'Error', )
            plt.ylabel(metric, fontsize = 13)
            plt.xlabel('Model', fontsize = 13)
            #plt.title(metric + ' for Eight Supervised Learning Models', fontsize = 15)
            for container in ax0.containers:
                ax0.bar_label(container)
            #plt.savefig('metric'+metric+'.jpg', dpi = 300)
            plt.show()
                                                                            

In [ ]:
tf.random.set_seed(2000)
load_mod = tf.keras.models.load_model('Saved_Neural_Nets/best_power.h5')
#load_mod.fit(x_train_scaled, y_traina, epochs=50, batch_size= 32)

In [ ]:
#np.sqrt(mean_squared_error(y_testa, load_mod.predict(x_test_scaled)))

In [ ]:
# best models map
best_regressors = {
    'LGBM': best_lgbm,
    'XGB':best_xgb,
    'RandomForest': best_rf,
    'GradientBoosting': best_grad,
    'Bagging': best_bag,
    'SVR': best_svr,
    'KNN': best_knn,
    'ANN': load_mod
}

In [ ]:
total_report(best_regressors, X_traina, X_testa, X_train, X_test, x_train_scaled, x_test_scaled, y_traina, y_testa)

In [ ]:
load_mod.summary()

# Conclusion

1. An ANN with 4 fully connected layers with the relu activation function gave the lowest results of rmse of 0.059, mae of	0.037, r2 score of 1.000 and rrmse of 0.431.
2. SVR came second with rmse of 0.065, mae of 0.055, r2 score of 1 and RRMSE of 0.474 on the test set.
3. The training and testing speed recorded here is from a laptop running on intel core i5  @2.4ghz clock speed with quadcore processor and 16gb ram. Definitely a GPU or a more advanced system with better processing power will train faster.
3. Wind speed was more important than ambient temp used together with solar irradiance (only these two features were used for all models) I think because it provided more space in the dimensional plot. Tree based models basically do logical splits and then take a mean. For categories found using wind speed with irradiance, there's freer points. See [EDA above](#above)
5. The tuned KNN, bagging, gradient boosting and random forest overfitted on the training set.
6. LGBM had the highest training time followed by the ANN.

### Resizing Images

In [ ]:
direc = os.listdir('predict')
for img in direc:
        # Here, I open all pictures, resize and save  
    image = Image.open(img)
    print('Resizing '+img+ '...')
    new_image = image.resize((4667, 3500))
    new_image.save('resized_'+img, dpi = (300, 300))
    print('Completed \n')

<a name="eff"></a>
## 15. EDA for Efficiency feature

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Solar_Irradiance', y='N_PV-TE', data = antarctica_df)
plt.grid()

Observation

1. We notice that from around 800 w/m2 of solar Irradiance, the values remain constant, basically there's a limit to the efficiency.
2. When solar Irradiance is 0, that's the only time there's 0 efficiency as long as Irradiance takes a value > 0 there's always efficiency.
3. Minimum efficiency after 0 is around 10 so for all working panels, there's a minimum efficiency of around 10%
4. The 0 days are actually outliers.

In [ ]:
# Doing some analysis on 0 days
antarctica_df[antarctica_df['Solar_Irradiance'] <= 0]

In [ ]:
antarctica_df[np.logical_and(antarctica_df['Solar_Irradiance'] <= 0.1, antarctica_df['N_PV-TE'] > 0)]

When solar irradiance is 0, efficiency is always 0, so 2021-08-25 is an outlier day. (This data was already removed from the dataset)

<a name="logt"></a>
### 16. Log transform on Solar Irradiance

In [ ]:
# The solar Irradiance distribution is skewed it will affect linear models
plt.hist(antarctica_df[['Solar_Irradiance']], bins=30)

In [ ]:
def log(val):
    return math.log(val, 10)

# log transforming solar irrad
antarctica_df['new_solar'] = antarctica_df.Solar_Irradiance+1
antarctica_df['irrad_log'] = antarctica_df.new_solar.apply(log)


In [ ]:
plt.hist(antarctica_df[['irrad_log']], bins=30)

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Ambient_Temp', y='N_PV-TE', data = antarctica_df)

Observation
1. Unlike in Solar Irradiance, the efficiency can take a value of 0 even when the ambient temperature isn't 0.

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Wind_Speed', y='N_PV-TE', data = antarctica_df)

From these we see that Solar Irradiance is a Key feature towards predicting both power output and efficiency

<a name = 'sect' a></a>
## EDA with Efficiency Binned

In [ ]:
antarctica_df.groupby('eff_bin')['N_PV-TE'].mean()

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Ambient_Temp', y='irrad_log', data = antarctica_df, hue='eff_bin')

In [ ]:
plt.figure(figsize= (12,7))
sns.scatterplot(x='Wind_Speed', y='irrad_log', data = antarctica_df, hue='eff_bin')

# Why KNN will perform poorly 
Using any of the other features with solar irrad, knn seems to be horizontally binned now, if we scale both y and x axis knn won't realize that it should take horizontal closeness. To make the nearest neighbours considered horizontally, that could be achieved by scaling the wind speed axis and leaving the Solar axis unscaled.

In [ ]:
# Comparing Seasonality and stationarity of Irradiance to the target Variable

sub = antarctica_df.copy()
sub = sub[['Date', 'Solar_Irradiance', 'N_PV-TE']]
sub.set_index('Date', inplace= True)
sub.plot(subplots = True, title = 'Solar Irradiance(w/m²) and P', figsize = (12,10))


Observations
1. The Values are seasonal and stationary although we see that there are no varying values (no roughness in plot) after 0 till around 10. But the peaks occur at similar times. Again this explains the relationship we saw earlier

In [ ]:
# Comparing Seasonality of Temperature to the target Variable

sub = antarctica_df.copy()
sub = sub[['Date', 'Ambient_Temp', 'N_PV-TE']]
sub.set_index('Date', inplace= True)
sub.plot(subplots = True, title = 'Temperature and P', figsize = (12,10))

In [ ]:
# Comparing trend of Wind Speed to the target Variable

sub = antarctica_df.copy()
sub = sub[['Date', 'Wind_Speed', 'N_PV-TE']]
sub.set_index('Date', inplace= True)
sub.plot(subplots = True, title = 'Wind_Speed and P', figsize = (12,10))


<a name="outlier" ></a>
### 17. Outlier detection and removal
(This was handled above) [here](#outlier_handled)

In [ ]:
# I scaled down solar Irradiance to make the plot below
antarctica_df['irr_scaled'] = antarctica_df.Solar_Irradiance/(antarctica_df.Solar_Irradiance.max()/14)+0.1

In [ ]:
# day of year and efficiency
plt.figure(figsize= (12,7))
sns.scatterplot(x='dayofyear', y='N_PV-TE', data = antarctica_df)

# scaling down the Solar Irradiance to be between 0-10 to make good comparison with efficiency
sns.scatterplot(x='dayofyear', y='irr_scaled', data = antarctica_df)
plt.grid()

Observations
1. The plot above brings out the relationship of efficiency each year, this is basically the diagram at the peaks we see that at the last 60 days of the year and the first 40 days, we have a constant efficiency and that's also the period where irradiance is highest. And when irradiance drops to 0, efficiency drops to 0
2. From the plots above, we notice an outlier at around august 2021, the efficiency value was higher than usual, I'll be removing that.

In [ ]:
# on inspection above I noticed it was on the 25th August
antarctica_df = antarctica_df[antarctica_df.Date != '2021-08-25']

In [ ]:
# replotting
plt.figure(figsize= (12,7))
sns.scatterplot(x='dayofyear', y='N_PV-TE', data = antarctica_df)

sns.scatterplot(x='dayofyear', y='irr_scaled', data = antarctica_df)
plt.grid()

In [ ]:
# Correlation Heatmap
corr = antarctica_df.corr()
plt.figure(figsize = (13, 8))
sns.heatmap(corr, cmap='RdYlGn', annot = True, center = 0)
plt.title('Correlogram', fontsize = 15, color = 'darkgreen')
plt.show()

Observations
1. We notice the highest correlation of efficiency with log transform of Solar Irradiance 

In [ ]:
antarctica_dfe = antarctica_df.copy()[['irrad_log', 'Wind_Speed', 'Ambient_Temp', 'dayofyear', 'weekofyear', 'month', 'day',
                                       'weekday', 'N_PV-TE', 'Date']]

In [ ]:
n_train = antarctica_dfe.shape[0] - round(antarctica_dfe.shape[0]*0.333)
n_train

In [ ]:
X_train1 = antarctica_dfe.drop(['N_PV-TE'], axis= 1).iloc[:n_train, :]
y_traine = antarctica_dfe[['N_PV-TE']].iloc[:n_train, :]

X_test1 = antarctica_dfe.drop(['N_PV-TE'], axis= 1).iloc[n_train:, :]
y_teste = antarctica_dfe[['N_PV-TE']].iloc[n_train:, :]

In [ ]:
X_traine = X_train1.drop('Date', axis = 1)
X_teste = X_test1.drop('Date', axis = 1)

In [ ]:
X_test1.Date.min()

In [ ]:
X_traine, X_teste = scale_datasets(X_traine, X_teste, kind ='min')

In [ ]:
regressors_e = {
    'LGBM': LGBMRegressor(random_state=1),
    'Linear_regression':LinearRegression(),
    'Ridge': Ridge(),
    'XGB': XGBRegressor(random_state = 66),
    'cat': CatBoostRegressor(random_state = 77),
    'RandomForest': RandomForestRegressor(random_state = 99),
    'Decision Tree': DecisionTreeRegressor(random_state = 12),
    'Adaboost': AdaBoostRegressor(random_state= 40),
    'Gboosting': GradientBoostingRegressor(random_state= 42)
}
for mods in regressors_e:
    recurse(X_traine, y_traine, regressors_e[mods])

In [ ]:
# manually creating something similar for SVR and KNN since svr in the rbf kernel doesnt provide coef_ or 
# feature importances
models = {'SupportVector':SVR(), 'KNN': KNeighborsRegressor(), 
         'LGBM': LGBMRegressor(random_state=1),
    'XGB': XGBRegressor(random_state = 66)
}

columns = X_teste.columns
scores = []
cols = []
test_scores = []

for model in models:
    print('FOR MODEL '+model)
    for switch in range(len(columns)):
        if switch>=0 and switch<=len(columns)-1:
            current_cols = list(columns)[:switch+1]
            #print('training on columns', current_cols)
            cross_val = cross_val_score(models[model], X_traine[current_cols], y_traine, scoring = 'neg_mean_squared_error',
                                                      cv= 5, verbose= False, n_jobs=-1)
            #print('Using {} mean rmse is {}'.format(current_cols, np.mean(cross_val*-1)))
            models[model].fit(X_traine[current_cols], y_traine)
            pred = models[model].predict(X_teste[current_cols])
            test_scores.append(np.sqrt(mean_squared_error(y_teste, pred)))
            scores.append(np.mean(cross_val*-1))
            cols.append(current_cols)
            
        else:
            break
    print('For model, we had the lowest crossval score of {} with {}'.format(min(scores), cols[scores.index(min(scores))]))
    print('For model, we had the lowest test score of {} with {}'.format(min(test_scores), 
                                                                         cols[test_scores.index(min(test_scores))]))
    print(test_scores)
    #print('trained on columns', cols, scores)
    print('\n')
    scores = []
    cols = []
    test_scores = []

Summary

From the results above, it makes sense why linear models like linear regression and ridge regression take ambient temp as an important feature as we have a high positive correlation of the target with ambient temperature. But from the visualizations, we see that no feature relates very linear with the target variable. So it makes sense not to listen to the linear models and listen instead to tree based models.

In doing that, we see that irradiance log transfromed (because of the skewness caused by 0 efficiency days) and Wind speed are the most important features. 

<a name = 'model_eff'></a>
## 18. Modelling

In [ ]:
 # Using just two Features
X_traine = X_traine[['irrad_log', 'Wind_Speed']]

X_teste = X_teste[['irrad_log', 'Wind_Speed']]

In [ ]:
train_report(X_traine, X_teste, y_traine, y_teste)

<a name = 'model_tun'></a>
## 19. Model Tuning
Tuning the Top 5 models except catboost along with KNN, Support Vector and an ANN. 

### 1. LGBMRegressor
I actually tuned a lot of Values for each parameter but I narrowed it down to this.

In [ ]:
lgbm_params_e = {'boosting_type':['dart'], 'max_depth':[-1], 'num_leaves':[6], 
               'learning_rate':[0.25,0.3], 'n_estimators':[2000,2100], 'reg_lambda':[0],
                 'reg_alpha':[0], 'colsample_bytree': [0.9], 'min_child_samples': [5, 6]}

In [ ]:
grid_lgb_e = GridSearchCV(LGBMRegressor(seed= model_seed), param_grid = lgbm_params_e, scoring= 'neg_root_mean_squared_error', cv= 4, n_jobs = -1, 
                       verbose= 10)

In [ ]:
%%time
grid_lgb_e.fit(X_traine, y_traine)

In [ ]:
grid_lgb_e.best_params_

In [ ]:
grid_lgb_e.best_score_

### Tuning History

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.25,
 'max_depth': -1,
 'min_child_samples': 5,
 'n_estimators': 2100,
 'num_leaves': 6,
 'reg_alpha': 0,
 'reg_lambda': 0}

-0.021351897782306173

{'boosting_type': 'dart',
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': -1,
 'min_child_samples': 5,
 'n_estimators': 2100,
 'num_leaves': 5,
 'reg_alpha': 0,
 'reg_lambda': 0}

-0.022409586935143628

### Getting Final results

In [ ]:
best_lgbm_e = grid_lgb_e.best_estimator_

In [ ]:
lgb_mape = {'Untuned model':LGBMRegressor(), 'Tuned': best_lgbm_e}

In [ ]:
compare_tuned(lgb_mape, X_traine, X_teste, y_traine, y_teste)

We notice an improvement on the tuned LGBM on the test data wrt the metrics. And looking at feature importance below, we notice that the tuned one takes iraddiance as far more important than other features which makes sense.

In [ ]:
plot_feat_imp(X_traine, best_lgbm_e)

In [ ]:
plot_real_pred(X_teste, y_teste, best_lgbm_e)

### 2. XGB Regressor

In [ ]:
xgb_params_e = {'n_estimators':[1000, 1200], 'booster':['gbtree'], 'max_depth':[4, 5], 
              'max_leaves':[0], 'colsample_bynode': [1],
               'learning_rate':[0.01,0.05,0.06], 'reg_lambda':[0], 'gamma':[0], 'reg_alpha':[0],
               'colsample_bytree': [1]}

In [ ]:
grid_xgb_e = GridSearchCV(XGBRegressor(random_state =model_seed), param_grid = xgb_params_e, scoring= 'neg_root_mean_squared_error', cv= 4, n_jobs = -1, 
                       verbose= 10)

In [ ]:
%%time
grid_xgb_e.fit(X_traine, y_traine)

In [ ]:
grid_xgb_e.best_params_ 

In [ ]:
grid_xgb_e.best_score_

### Tuning History

{'booster': 'gbtree',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.05,
 'max_depth': 4,
 'max_leaves': 0,
 'n_estimators': 800,
 'reg_alpha': 0,
 'reg_lambda': 0}

-0.029882937472050383

{'booster': 'gbtree',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.08,
 'max_depth': 4,
 'max_leaves': 0,
 'n_estimators': 600,
 'reg_alpha': 0,
 'reg_lambda': 0}

-0.03058499139479354

{'booster': 'dart',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.05,
 'max_depth': 4,
 'max_leaves': 0,
 'n_estimators': 2000,
 'reg_alpha': 0,
 'reg_lambda': 0}

-0.029883656245360234

{'booster': 'dart',
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.03,
 'max_depth': 4,
 'max_leaves': 0,
 'n_estimators': 2000,
 'reg_alpha': 0,
 'reg_lambda': 0}
 
-0.0299683327936906

### Getting Final results

In [ ]:
best_xgb_e = grid_xgb_e.best_estimator_

In [ ]:
xgb_mape= {'Untuned model':XGBRegressor(), 'Tuned': best_xgb_e}

In [ ]:
compare_tuned(xgb_mape, X_traine, X_teste, y_traine, y_teste)

In [ ]:
plot_feat_imp(X_traine, best_xgb_e)

In [ ]:
plot_real_pred(X_teste, y_teste, best_xgb_e)

## 3. Random Forest Regressor

In [ ]:
rf_params_e = {'n_estimators':[300,600], 'max_depth':[None], 'min_samples_split': [2, 3],
             'min_samples_leaf': [1, 2], 'max_leaf_nodes':[None]}

In [ ]:
grid_rf_e = GridSearchCV(RandomForestRegressor(random_state=model_seed), param_grid = rf_params_e, scoring= 'neg_root_mean_squared_error', cv= 4,
                       n_jobs = -1, verbose= 10)

In [ ]:
%%time
grid_rf_e.fit(X_traine, y_traine)

In [ ]:
grid_rf_e.best_params_

In [ ]:
grid_rf_e.best_score_

### Tuning History

{'max_depth': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600}

-0.1153784572102072

{'max_depth': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 700}

-0.11723652477574889

### Getting Final results

In [ ]:
best_rf_e = grid_rf_e.best_estimator_

In [ ]:
rf_mape = {'Untuned model':RandomForestRegressor(), 'Tuned': best_rf_e}

In [ ]:
compare_tuned(rf_mape, X_traine, X_teste, y_traine, y_teste)

There's no visible improvement with Random Forest

In [ ]:
plot_feat_imp(X_traine, best_rf_e)

## 4.Bagging Regressor

In [ ]:
bag_params_e = {'n_estimators':[190,200],'max_samples':[0.8, 0.9, 1],
               'warm_start': [False], 'max_features':[1,2]}

In [ ]:
grid_bag_e = GridSearchCV(BaggingRegressor(random_state =model_seed), param_grid = bag_params_e,
                         scoring= 'neg_root_mean_squared_error', cv= 4,
                       n_jobs = -1, verbose= 10)

In [ ]:
%%time
grid_bag_e.fit(X_traine, y_traine)

In [ ]:
grid_bag_e.best_params_

In [ ]:
grid_bag_e.best_score_

### Tuning History

### Getting Final results

In [ ]:
best_bag_e = grid_bag_e.best_estimator_

In [ ]:
bag_mape = {'Untuned model':BaggingRegressor(), 'Tuned': best_bag_e}

In [ ]:
compare_tuned(bag_mape, X_traine, X_teste, y_traine, y_teste)

There's no much improvement with bagging regressor

In [ ]:
plot_real_pred(X_teste, y_teste, best_bag_e)

## 5. Gradient Boosting Regressor

In [ ]:
grad_params_e = {'n_estimators':[3000], 'criterion':['squared_error'], 'min_samples_split':[2], 
              'min_samples_leaf':[2], 'max_depth': [3,4], 'max_leaf_nodes': [None],
               'learning_rate':[0.07]}

In [ ]:
grid_grad_e = GridSearchCV(GradientBoostingRegressor(random_state = model_seed), param_grid = grad_params_e, 
                          scoring= 'neg_root_mean_squared_error', cv= 5,
                       n_jobs = -1, verbose= 10)

In [ ]:
%%time
grid_grad_e.fit(X_traine, y_traine)

In [ ]:
grid_grad_e.best_params_

In [ ]:
grid_grad_e.best_score_ 

### Tuning History

{'criterion': 'squared_error',
 'learning_rate': 0.07,
 'max_depth': 4,
 'max_leaf_nodes': None,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 2000}

-0.029082164170848655

{'criterion': 'squared_error',
 'learning_rate': 0.05,
 'max_depth': 3,
 'max_leaf_nodes': None,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 1500}

-0.029612643004875656

### Getting Final results

In [ ]:
best_grad_e = grid_grad_e.best_estimator_

In [ ]:
grad_mape = {'Untuned model':GradientBoostingRegressor(), 'Tuned': best_grad_e}

In [ ]:
compare_tuned(grad_mape, X_traine, X_teste, y_traine, y_teste)

In [ ]:
plot_feat_imp(X_traine, best_grad_e)

In [ ]:
plot_real_pred(X_teste, y_teste, best_grad_e)

## 6. Support Vector Machine

In [ ]:
x_train_scaledm, x_test_scaledm = scale_datasets(X_train1[['irrad_log', 'Wind_Speed']], X_test1[['irrad_log', 'Wind_Speed']]
                                                 , kind ='min')
x_train_scaled_e, x_test_scaled_e = scale_datasets(X_train1[['irrad_log', 'Wind_Speed']], X_test1[['irrad_log', 'Wind_Speed']])

### MinMax scaler performed better than Standard Scaler for SVM

In [ ]:
svr_params_e = {'kernel':['rbf'], 'degree':[1], 'C':[5000, 7000, 10000], 
                'gamma':[7, 10, 15]} 
              

In [ ]:
grid_svr_e = GridSearchCV(SVR(), param_grid = svr_params_e, scoring= 'neg_root_mean_squared_error', cv= 5, n_jobs = -1, 
                       verbose= 10)

In [ ]:
%%time
grid_svr_e.fit(X_traine, y_traine)

In [ ]:
grid_svr_e.best_params_ 

In [ ]:
grid_svr_e.best_score_

#### Tuning History

{'C': 5000, 'degree': 1, 'gamma': 15, 'kernel': 'rbf'}

-0.5240788583595232

{'C': 2000, 'degree': 1, 'gamma': 10, 'kernel': 'rbf'}

-0.5435537263935626

{'C': 1500, 'degree': 1, 'gamma': 20, 'kernel': 'rbf'}

-0.5476583102447977

{'C': 120, 'degree': 1, 'gamma': 30, 'kernel': 'rbf'}

-0.5912933693634681

{'C': 100, 'degree': 1, 'gamma': 100, 'kernel': 'rbf'}

-0.6507738704020458

### Getting Final results

In [ ]:
best_svr_e = grid_svr_e.best_estimator_

In [ ]:
svr_mape = {'Untuned model':SVR(), 'Tuned': best_svr_e}

In [ ]:
compare_tuned(svr_mape, X_traine, X_teste, y_traine, y_teste)

We notice good improvement

In [ ]:
plot_real_pred(x_test_scaledm, y_teste, best_svr_e)

# 7. KNN Regressor

In [ ]:
knn = KNeighborsRegressor()

In [ ]:
# Using only irrad_log on KNN
knn.fit(x_train_scaledm[['irrad_log']], y_traine)

In [ ]:
np.sqrt(mean_squared_error(y_teste, knn.predict(x_test_scaledm[['irrad_log']])))

In [ ]:
knn_params_e = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10], 'weights':['uniform', 'distance'], 'p':[4,5,6,7,8,9], 
             'metric': ['euclidean', 'manhattan', 'minkowski']} 
              

In [ ]:
grid_knn_e = GridSearchCV(KNeighborsRegressor(), param_grid = knn_params_e, scoring= 'neg_root_mean_squared_error', cv= 4, n_jobs = -1, 
                       verbose= 10)

In [ ]:
%%time
grid_knn_e.fit(X_traine, y_traine)

In [ ]:
grid_knn_e.best_params_ 

In [ ]:
grid_knn_e.best_score_

### Getting Final results

In [ ]:
best_knn_e = grid_knn_e.best_estimator_

In [ ]:
knn_mape = {'Untuned model':KNeighborsRegressor(), 'Tuned': best_knn_e}

In [ ]:
compare_tuned(knn_mape, X_traine, X_teste, y_traine, y_teste)

In [ ]:
plot_real_pred(X_teste, y_teste, best_knn_e)

# Why KNN performs poorly using the wind and ambient temp features with solar Irrad as compared to Irradiance alone
The efficiency value was binned into lowest, lower...peak above. See [EDA section](#sect) above for the plot.

First of all the difference between efficiency values in the lowest(mostly 0s) and lower region is much. So if just one point that should be in the lowest category gets into the lower category the error will be raised.

When irradiance values are slightly >0, we notice that there's efficiency, but now if we have a point really close to 0 like we actually do, we notice that considering the nearest neighbour using temperature or ambient temp on 2d plane, we'd have cases where points close to 0 will have nearest neighbours closer to the lowest region instead of them being in the lower region

**On training using only Irrad log**
- untuned KNN - rmse = 0.07

**with other features**
- untuned KNN - rmse = 0.528

# 8. Artificial Neural Network (ANN)

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  tf.keras.Input(shape=(2,))

  
  for i in range(hp.Int('layer', 2, 15)):
    model.add(Dense(hp.Int('units_'+ str(i), min_value=32, max_value=512, step=32), kernel_initializer='normal',
                    activation='relu'))  

  hp_learning_rate = hp.Choice('learning_rate', values=[0.0001, 0.001, 0.003, 0.005])
  
  model.add(Dense(1, kernel_initializer='normal', activation='linear'))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.MeanSquaredError(),
                metrics=['mse'])

  return model

In [ ]:
# max trials just says how many combination of hyperparameters should I try, them executions say for each combination, how
# many times should I train on that ? Because for NNs we get different results for the same model

tuner = kt.RandomSearch(model_builder,
                     objective=kt.Objective("val_mse", direction="min"),
                     max_trials=50,
                     executions_per_trial=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

In [ ]:
tuner.search(x_train_scaled.values, y_traine.values, epochs=50, validation_split=0.2, callbacks=[stop_early],
             batch_size =64)

# Getting the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
tuner.results_summary()

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# Building the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train_scaled, y_traine, epochs=60, validation_split=0.2, batch_size =32)

val_acc_per_epoch = history.history['val_mse']
best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retraining the model
hypermodel.fit(x_train_scaled, y_traine, epochs=best_epoch, validation_split=0.2, batch_size = 32)

In [ ]:
eval_result = hypermodel.evaluate(x_test_scaled, y_teste)
print("[test loss, test mse]:", eval_result)

In [ ]:
hypermodel.summary()

In [ ]:
#hypermodel.save('best_efficiency.h5')

In [ ]:
tf.random.set_seed(20)

load_mod_e = tf.keras.models.load_model('Saved_Neural_Nets/best_efficiency.h5')
#load_mod_e.fit(x_train_scaled, y_traine, epochs=3, batch_size= 64)

In [ ]:
#np.sqrt(mean_squared_error(y_teste, load_mod_e.predict(x_test_scaled)))

<a name = 'summary_eff'></a>
## 20. Summary and Conclusions

In [ ]:
# best models map
best_regressors_e = {
    'LGBM': best_lgbm_e,
    'XGB':best_xgb_e,
    'RandomForest': best_rf_e,
    'GradientBoosting':best_grad_e,
    'Bagging': best_bag_e,
    'SVR': best_svr_e,
    'KNN': best_knn_e,
    'ANN': load_mod_e
}

In [ ]:
def visualize_pred(X_train, X_test, y_train, y_test, train_pr, test_pr, model):
    plt.figure(figsize = (12,8))
    plt.plot(X_train.Date, y_train, color = 'b', label = 'Real')
    plt.plot(X_train.Date, train_pr, color = 'r', label = model)
    plt.text(pd.to_datetime('04/25/2019'),13, 'Training Data', fontsize = 13)
    plt.text(pd.to_datetime('04/25/2021'),13, 'Testing Data', fontsize = 13)


    plt.plot(X_test.Date, y_test, color = 'b')
    plt.plot(X_test.Date, test_pr, color = 'r')
    plt.axvline(X_test.Date.min(), color= 'k', ls = '--')
    plt.xlabel('Date', fontsize = 14)
    plt.ylabel('Efficiency (%)', fontsize = 14)
    
    plt.legend()
    #plt.savefig('EFF_pred'+model+'.jpg', dpi = 300)
    plt.show()
    

In [ ]:
def total_report(model_map, X_traindate, X_testdate, X_train, X_test, X_trainsc, X_testsc, y_train, y_test ):
    df_models = pd.DataFrame(columns=['model', 'MSE', 'RMSE', 'MAE', 'r2', 'mape', 'RRMSE', 'Error'])

    for a, key in enumerate(model_map):

        start_time = time.time()
        
        if key == 'ANN':
            regressor = model_map[key]
            #X_trainsc, X_testsc = scale_datasets(X_train, X_test)
            #tf.random.set_seed(200)
            start_time = time.time()
            regressor.fit(X_trainsc, y_train, epochs=3, batch_size= 64)
            end_time = time.time()
            y_pred_tr = regressor.predict(X_trainsc)
            start_time_pr = time.time()
            y_pred_te = regressor.predict(X_testsc)
            end_time_pr = time.time()
        else:
            regressor = model_map[key]
            start_time = time.time()
            regressor.fit(X_train, y_train)
            end_time = time.time()
            y_pred_tr = regressor.predict(X_train)
            start_time_pr = time.time()
            y_pred_te = regressor.predict(X_test)
            end_time_pr = time.time()
        # metrics
        train_time = end_time - start_time
        pr_time = end_time_pr - start_time_pr
        #print(pr_time, train_time)
        #tr_te = ['train', 'test']
        rmse = [np.sqrt(mean_squared_error(y_train, y_pred_tr)),  np.sqrt(mean_squared_error(y_test, y_pred_te))] 
        mse = [mean_squared_error(y_train, y_pred_tr), mean_squared_error(y_test, y_pred_te)]
        mae = [mean_absolute_error(y_train, y_pred_tr), mean_absolute_error(y_test, y_pred_te)]
        r2 = [r2_score(y_train, y_pred_tr), r2_score(y_test, y_pred_te)]
        mape = [mean_absolute_percentage_error(y_train, y_pred_tr), mean_absolute_percentage_error(y_test, y_pred_te)]
        rrmse = [np.sqrt((mean_squared_error(y_train, y_pred_tr)/mean_squared_error(y_train, 
                                                                                    [np.mean(y_train)]*len(y_train))))*100,
                 np.sqrt((mean_squared_error(y_test, y_pred_te)/mean_squared_error(y_test, [np.mean(y_test)]*len(y_test))))*100]

        
        row_tr = {'model': key,
           'Time_trte': train_time,
            'MSE':mse[0],
           'RMSE': rmse[0],
            'MAE':mae[0],
            'r2': r2[0],
            'mape': mape[0] ,
            'RRMSE': rrmse[0],
            'Error': 'Train',
            'Time': 'Training'
        }
                      
        row_te = {'model': key,
           'Time_trte': pr_time,
            'MSE':mse[1],
           'RMSE': rmse[1],
            'MAE':mae[1],
            'r2': r2[1],
            'mape': mape[1] ,
            'RRMSE': rrmse[1],
            'Error': 'Test',
             'Time': 'Testing'
        }
        visualize_pred(X_traindate, X_testdate, y_train, y_test, y_pred_tr, y_pred_te, key)
        df_models = df_models.append(row_tr, ignore_index=True)
        df_models = df_models.append(row_te, ignore_index=True)
                                                                    
    # dispaying the top 5 models wrt rmse
    sorted_df = df_models.sort_values(['RMSE'], ascending = False).iloc[:,:].reset_index(drop = True)
    sorted_df = round(sorted_df, 3)
    display(sorted_df)
    display_metrics = ['MSE', 'RMSE', 'MAE', 'RRMSE', 'Time_trte']
    for metric in display_metrics:
        if metric == 'Time_trte':
            palette = {'Training':'red', 'Testing':'green'}
            sorted_df = df_models.sort_values(['Time_trte', 'Time'], ascending = False).iloc[:,:].reset_index(drop = True)
            sorted_df = round(sorted_df, 3)
            plt.figure(figsize = (12,8))
            ax0 = sns.barplot(x= 'model', y=metric, data= sorted_df, hue= 'Time', palette= palette ) 
            plt.ylabel('Time (s)', fontsize = 13)
            plt.xlabel('Model', fontsize = 13)
            plt.title('Training and Testing Time for Eight Supervised Learning Models', fontsize = 15)
            for container in ax0.containers:
                ax0.bar_label(container)
            #plt.savefig('metric'+metric+'.jpg', dpi = 300)
            plt.show()
        else:
            #palette = {'Train':'orange', 'Test':'blue'}
            plt.figure(figsize = (12,8))
            ax0 = sns.barplot(x= 'model', y=metric, data= sorted_df, hue= 'Error', hue_order= ['Test', 'Train'])  
            plt.ylabel(metric, fontsize = 13)
            plt.xlabel('Model', fontsize = 13)
            plt.title(metric + ' for Eight Supervised Learning Models', fontsize = 15)
            for container in ax0.containers:
                ax0.bar_label(container)
            #plt.savefig('Eff_metric'+metric+'.jpg', dpi = 300)
            plt.show()
                                                               

In [ ]:
total_report(best_regressors_e, X_train1, X_test1, X_traine, X_teste, x_train_scaled_e, x_test_scaled_e, y_traine, y_teste)

In [ ]:
def train_test(model, x_train, x_test):
    pred_train, pred_test = model.predict(x_train), model.predict(x_test)
    return pred_train, pred_test

In [ ]:
def all_together(model_map_pow, model_map_eff, X_train, X_test, x_train, x_test, x_traine, x_teste,
                 X_trainsc, X_testsc, X_trainsce, X_testsce, y_train_pow, y_test_pow, y_train_eff, y_test_eff):
    '''Plots real and predicted values of both Efficiency and Power on the same plot'''
    
    for key_pow, key_eff in zip(model_map_pow, model_map_eff):        
        if key_pow == 'ANN':
            regressor_pow = model_map_pow[key_pow]
            regressor_eff = model_map_eff[key_eff]
            regressor_pow.fit(X_trainsc, y_train_pow, epochs=50, batch_size= 32) 
            regressor_eff.fit(X_trainsce, y_train_eff, epochs=3, batch_size= 64)
            y_pred_tr_pow, y_pred_te_pow = train_test(regressor_pow, X_trainsc, X_testsc)
            y_pred_tr_eff, y_pred_te_eff = train_test(regressor_eff, X_trainsce, X_testsce)
        else:
            regressor_pow = model_map_pow[key_pow]
            regressor_eff = model_map_eff[key_eff]
            regressor_pow.fit(x_train, y_train_pow) 
            regressor_eff.fit(x_traine, y_train_eff)
            y_pred_tr_pow, y_pred_te_pow = train_test(regressor_pow, x_train, x_test)
            y_pred_tr_eff, y_pred_te_eff = train_test(regressor_eff, x_traine, x_teste)
            
    # Plotting real and predicted power and efficiency on the same axis
        fig, ax = plt.subplots(figsize = (12,8))
        ax1 = ax.twinx()

        ax.scatter(X_train.Date, y_pred_tr_pow, color= 'r', marker = '^', s= 45, label = key_pow+' (W)')
        ax.plot(X_train.Date, y_train_pow, color= 'r', label = 'Power (W)')
        ax.scatter(X_test.Date, y_pred_te_pow, color= 'r', marker = '^', s= 45)
        ax.plot(X_test.Date, y_test_pow, color= 'r')

        ax1.scatter(X_train.Date, y_pred_tr_eff, color = 'b', marker = '.', s= 45, label = key_pow+' (%)')
        ax1.plot(X_train.Date, y_train_eff, color = 'b', label = 'Efficiency (%)')
        ax1.scatter(X_test.Date, y_pred_te_eff, color = 'b', marker = '.', s= 45)
        ax1.plot(X_test.Date, y_test_eff, color = 'b')

        ax.set_xlabel('Date', fontsize = 13)
        ax.set_ylabel('Power (W)', fontsize = 13,color = 'r')
        ax.tick_params(axis='y', labelcolor = 'r')

        ax1.set_ylabel('Efficiency (%)', fontsize = 13, color = 'b')
        ax1.tick_params(axis='y', labelcolor = 'b')

        # Setting position of legend
        fig.legend(loc= (0.44, 0.42))
        #fig.legend(loc= 'upper center', bbox_to_anchor=(0.5, 0., 0.5, 0.5))
        
        plt.text(pd.to_datetime('04/25/2019'),13, 'Training Data', fontsize = 13)
        plt.text(pd.to_datetime('04/25/2021'),13, 'Testing Data', fontsize = 13)
        plt.axvline(X_test.Date.min(), color= 'k', ls = '--')
        plt.grid()

        # Giving plot a title
        plt.title('Real and Predicted Power and Efficiency of Panels Overtime in Antarctica', fontsize = 14)
        plt.savefig('power_and_efficiency'+key_pow+'.jpg', dpi = 300)

In [ ]:
all_together(best_regressors, best_regressors_e, X_train1, X_test1, X_train, X_test, X_traine, X_teste,
             x_train_scaled, x_test_scaled,x_train_scaled_e, x_test_scaled_e, y_traina, y_testa, y_traine, y_teste)

# Conclusion

Generally tree based models performed better because they are able to handle the skew because they use splits.
1. Light Gradient Boosting Machine (LGBM) Regressor gave the best results with rmse of 0.021, mae of 0.012, r2 score of 1 and RRMSE of 0.352 on the test set.
2. Gradient Boosting Regressor came second with rmse of 0.027, mae of 0.014, r2 score of 1 and RRMSE of 0.448 on the test set. 
3. Looking at the predictions SVR underfitted and performed poorly on both train and test sets.
4. The ANN underfitted on the training set and hence produced poor results on the test set.
5. KNN overfitted on the training set.
5. LGBM had the highest training time of 3.5 seconds on a pc running on intel core I5 @2.4ghz clock speed with a quadcore processor and 16gb of ram.


**The reason for the much higher train error in the ANN and the support vectors seems to be because there's more train data hence more datapoints contributing to the error.**

### Resizing Images

In [ ]:
direc = os.listdir('predict')
for img in direc:
    image = Image.open(img)
    print('Resizing '+img+ '...')
    new_image = image.resize((4667, 3500))
    new_image.save('resized_'+img, dpi = (300, 300))
    print('Completed \n')